In [29]:
import os
import pandas as pd
import pickle
import shutil
import numpy as np
import seaborn as sns
from sklearn.datasets import load_files
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score


from PIL import ImageFile   
from sklearn.model_selection import train_test_split
from keras.preprocessing import image                  
from tqdm import tqdm

from keras.applications.vgg16 import VGG16

In [2]:
TEST_DIR = os.path.join(os.getcwd(),"imgs","test")
TRAIN_DIR = os.path.join(os.getcwd(),"imgs","train")
MODEL_PATH = os.path.join(os.getcwd(),"model","vgg16")
PICKLE_PATH = os.path.join(os.getcwd(),"pickle")
TEST_CSV = os.path.join(os.getcwd(),"csv_files","test.csv")
TRAIN_CSV = os.path.join(os.getcwd(),"csv_files","train.csv")

In [3]:
if not os.path.exists(TEST_DIR):
    print("Testing data does not exists")
if not os.path.exists(TRAIN_DIR):
    print("Training data does not exists")
if not os.path.exists(MODEL_PATH):
    print("Model path does not exists")
    os.makedirs(MODEL_PATH)
    print("Model path created")
else:
    shutil.rmtree(MODEL_PATH)
    os.makedirs(MODEL_PATH)
if not os.path.exists(PICKLE_PATH):
    os.makedirs(PICKLE_PATH)

In [4]:
data_train = pd.read_csv(TRAIN_CSV)
data_test = pd.read_csv(TEST_CSV)

In [5]:
labels_list = list(set(data_train['ClassName'].values.tolist()))
labels_id = {label_name:id for id,label_name in enumerate(labels_list)}
print(labels_id)
data_train['ClassName'].replace(labels_id,inplace=True)

labels = to_categorical(data_train['ClassName'])
print(labels.shape)

with open(os.path.join(PICKLE_PATH,"labels_list_vgg16.pkl"),"wb") as handle:
    pickle.dump(labels_id,handle)

{'c5': 0, 'c9': 1, 'c7': 2, 'c1': 3, 'c4': 4, 'c3': 5, 'c6': 6, 'c0': 7, 'c8': 8, 'c2': 9}
(22424, 10)


In [6]:
xtrain,xtest,ytrain,ytest = train_test_split(data_train.iloc[:,0],labels,test_size = 0.2,random_state=42)
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(64, 64))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [7]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 
# pre-process the data for Keras
train_tensors = paths_to_tensor(xtrain).astype('float32')/255 - 0.5
valid_tensors = paths_to_tensor(xtest).astype('float32')/255 - 0.5

100%|██████████| 4485/4485 [01:36<00:00, 46.55it/s]


# MODEL ARCHITECTURE

In [8]:
model = VGG16(include_top=False)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [9]:
train_vgg16 = model.predict(train_tensors,verbose=1)
valid_vgg16 = model.predict(valid_tensors,verbose=1)

4485/4485 [==============================] - 106s 24ms/step


In [10]:
print("Train shape",train_vgg16.shape)
print("Validation shape",valid_vgg16.shape)

Train shape (17939, 2, 2, 512)
Validation shape (4485, 2, 2, 512)


In [11]:
train_features = train_vgg16[0]
valid_features = valid_vgg16[0]

In [12]:
print("Train features shape",train_features.shape)
print("Validation features shape",valid_features.shape)

Train features shape (2, 2, 512)
Validation features shape (2, 2, 512)


# Model Architecture

In [13]:
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_features.shape))
VGG16_model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))

VGG16_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
_________________________________________________________________


In [14]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
plot_model(VGG16_model,to_file=os.path.join(os.getcwd(),"model","vgg16","model_distracted_driver_vgg16.png"),show_shapes=True,show_layer_names=True)

In [16]:
filepath = os.path.join(MODEL_PATH,"distracted-{epoch:02d}-{val_accuracy:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',period=1)
callbacks_list = [checkpoint]

In [17]:
model_history = VGG16_model.fit(train_vgg16,ytrain,validation_data = (valid_vgg16, ytest),epochs=400, batch_size=16, shuffle=True,callbacks=callbacks_list)

Train on 17939 samples, validate on 4485 samples
Epoch 1/400
17939/17939 [==============================] - 1s 80us/step - loss: 1.7462 - accuracy: 0.4577 - val_loss: 1.4421 - val_accuracy: 0.5761

Epoch 00001: val_accuracy improved from -inf to 0.57614, saving model to /home/abhinav/distracted_driver/model/vgg16/distracted-01-0.58.hdf5
Epoch 2/400
17939/17939 [==============================] - 1s 74us/step - loss: 1.2457 - accuracy: 0.6593 - val_loss: 1.1513 - val_accuracy: 0.6767

Epoch 00002: val_accuracy improved from 0.57614 to 0.67670, saving model to /home/abhinav/distracted_driver/model/vgg16/distracted-02-0.68.hdf5
Epoch 3/400
17939/17939 [==============================] - 1s 72us/step - loss: 1.0238 - accuracy: 0.7264 - val_loss: 0.9689 - val_accuracy: 0.7371

Epoch 00003: val_accuracy improved from 0.67670 to 0.73712, saving model to /home/abhinav/distracted_driver/model/vgg16/distracted-03-0.74.hdf5
Epoch 4/400
17939/17939 [==============================] - 1s 75us/step - l


Epoch 00031: val_accuracy improved from 0.88673 to 0.88807, saving model to /home/abhinav/distracted_driver/model/vgg16/distracted-31-0.89.hdf5
Epoch 32/400
17939/17939 [==============================] - 2s 84us/step - loss: 0.3305 - accuracy: 0.9107 - val_loss: 0.3763 - val_accuracy: 0.8919

Epoch 00032: val_accuracy improved from 0.88807 to 0.89186, saving model to /home/abhinav/distracted_driver/model/vgg16/distracted-32-0.89.hdf5
Epoch 33/400
17939/17939 [==============================] - 2s 87us/step - loss: 0.3264 - accuracy: 0.9109 - val_loss: 0.3734 - val_accuracy: 0.8919

Epoch 00033: val_accuracy did not improve from 0.89186
Epoch 34/400
17939/17939 [==============================] - 1s 83us/step - loss: 0.3223 - accuracy: 0.9119 - val_loss: 0.3680 - val_accuracy: 0.8945

Epoch 00034: val_accuracy improved from 0.89186 to 0.89454, saving model to /home/abhinav/distracted_driver/model/vgg16/distracted-34-0.89.hdf5
Epoch 35/400
17939/17939 [==============================] - 1s

17939/17939 [==============================] - 1s 76us/step - loss: 0.2480 - accuracy: 0.9327 - val_loss: 0.3091 - val_accuracy: 0.9072

Epoch 00066: val_accuracy did not improve from 0.90747
Epoch 67/400
17939/17939 [==============================] - 1s 76us/step - loss: 0.2476 - accuracy: 0.9318 - val_loss: 0.3046 - val_accuracy: 0.9101

Epoch 00067: val_accuracy improved from 0.90747 to 0.91014, saving model to /home/abhinav/distracted_driver/model/vgg16/distracted-67-0.91.hdf5
Epoch 68/400
17939/17939 [==============================] - 1s 76us/step - loss: 0.2458 - accuracy: 0.9320 - val_loss: 0.3039 - val_accuracy: 0.9099

Epoch 00068: val_accuracy did not improve from 0.91014
Epoch 69/400
17939/17939 [==============================] - 1s 76us/step - loss: 0.2446 - accuracy: 0.9328 - val_loss: 0.3063 - val_accuracy: 0.9088

Epoch 00069: val_accuracy did not improve from 0.91014
Epoch 70/400
17939/17939 [==============================] - 1s 75us/step - loss: 0.2434 - accuracy: 0.93

17939/17939 [==============================] - 2s 85us/step - loss: 0.1973 - accuracy: 0.9439 - val_loss: 0.2808 - val_accuracy: 0.9144

Epoch 00142: val_accuracy did not improve from 0.91706
Epoch 143/400
17939/17939 [==============================] - 1s 79us/step - loss: 0.1972 - accuracy: 0.9444 - val_loss: 0.2848 - val_accuracy: 0.9139

Epoch 00143: val_accuracy did not improve from 0.91706
Epoch 144/400
17939/17939 [==============================] - 1s 83us/step - loss: 0.1965 - accuracy: 0.9438 - val_loss: 0.2826 - val_accuracy: 0.9148

Epoch 00144: val_accuracy did not improve from 0.91706
Epoch 145/400
17939/17939 [==============================] - 1s 83us/step - loss: 0.1962 - accuracy: 0.9451 - val_loss: 0.2831 - val_accuracy: 0.9135

Epoch 00145: val_accuracy did not improve from 0.91706
Epoch 146/400
17939/17939 [==============================] - 1s 83us/step - loss: 0.1956 - accuracy: 0.9435 - val_loss: 0.2800 - val_accuracy: 0.9162

Epoch 00146: val_accuracy did not impro

17939/17939 [==============================] - 1s 76us/step - loss: 0.1871 - accuracy: 0.9472 - val_loss: 0.2823 - val_accuracy: 0.9175

Epoch 00181: val_accuracy did not improve from 0.91839
Epoch 182/400
17939/17939 [==============================] - 1s 75us/step - loss: 0.1870 - accuracy: 0.9467 - val_loss: 0.2823 - val_accuracy: 0.9162

Epoch 00182: val_accuracy did not improve from 0.91839
Epoch 183/400
17939/17939 [==============================] - 1s 75us/step - loss: 0.1863 - accuracy: 0.9467 - val_loss: 0.2822 - val_accuracy: 0.9164

Epoch 00183: val_accuracy did not improve from 0.91839
Epoch 184/400
17939/17939 [==============================] - 1s 75us/step - loss: 0.1865 - accuracy: 0.9487 - val_loss: 0.2869 - val_accuracy: 0.9142

Epoch 00184: val_accuracy did not improve from 0.91839
Epoch 185/400
17939/17939 [==============================] - 1s 75us/step - loss: 0.1862 - accuracy: 0.9473 - val_loss: 0.2836 - val_accuracy: 0.9155

Epoch 00185: val_accuracy did not impro

17939/17939 [==============================] - 1s 76us/step - loss: 0.1767 - accuracy: 0.9501 - val_loss: 0.2923 - val_accuracy: 0.9173

Epoch 00259: val_accuracy did not improve from 0.91906
Epoch 260/400
17939/17939 [==============================] - 1s 77us/step - loss: 0.1770 - accuracy: 0.9500 - val_loss: 0.2941 - val_accuracy: 0.9177

Epoch 00260: val_accuracy did not improve from 0.91906
Epoch 261/400
17939/17939 [==============================] - 1s 76us/step - loss: 0.1755 - accuracy: 0.9510 - val_loss: 0.2930 - val_accuracy: 0.9168

Epoch 00261: val_accuracy did not improve from 0.91906
Epoch 262/400
17939/17939 [==============================] - 1s 76us/step - loss: 0.1760 - accuracy: 0.9499 - val_loss: 0.2980 - val_accuracy: 0.9175

Epoch 00262: val_accuracy did not improve from 0.91906
Epoch 263/400
17939/17939 [==============================] - 1s 75us/step - loss: 0.1758 - accuracy: 0.9502 - val_loss: 0.2923 - val_accuracy: 0.9179

Epoch 00263: val_accuracy did not impro

17939/17939 [==============================] - 1s 76us/step - loss: 0.1740 - accuracy: 0.9508 - val_loss: 0.2998 - val_accuracy: 0.9171

Epoch 00299: val_accuracy did not improve from 0.91906
Epoch 300/400
17939/17939 [==============================] - 1s 76us/step - loss: 0.1741 - accuracy: 0.9511 - val_loss: 0.3018 - val_accuracy: 0.9128

Epoch 00300: val_accuracy did not improve from 0.91906
Epoch 301/400
17939/17939 [==============================] - 1s 77us/step - loss: 0.1738 - accuracy: 0.9513 - val_loss: 0.3011 - val_accuracy: 0.9157

Epoch 00301: val_accuracy did not improve from 0.91906
Epoch 302/400
17939/17939 [==============================] - 1s 77us/step - loss: 0.1739 - accuracy: 0.9509 - val_loss: 0.3052 - val_accuracy: 0.9153

Epoch 00302: val_accuracy did not improve from 0.91906
Epoch 303/400
17939/17939 [==============================] - 1s 77us/step - loss: 0.1734 - accuracy: 0.9515 - val_loss: 0.3009 - val_accuracy: 0.9177

Epoch 00303: val_accuracy did not impro

17939/17939 [==============================] - 1s 81us/step - loss: 0.1721 - accuracy: 0.9532 - val_loss: 0.3146 - val_accuracy: 0.9177

Epoch 00378: val_accuracy did not improve from 0.91929
Epoch 379/400
17939/17939 [==============================] - 2s 91us/step - loss: 0.1715 - accuracy: 0.9532 - val_loss: 0.3141 - val_accuracy: 0.9188

Epoch 00379: val_accuracy did not improve from 0.91929
Epoch 380/400
17939/17939 [==============================] - 2s 84us/step - loss: 0.1713 - accuracy: 0.9541 - val_loss: 0.3169 - val_accuracy: 0.9166

Epoch 00380: val_accuracy did not improve from 0.91929
Epoch 381/400
17939/17939 [==============================] - 1s 79us/step - loss: 0.1706 - accuracy: 0.9527 - val_loss: 0.3155 - val_accuracy: 0.9155

Epoch 00381: val_accuracy did not improve from 0.91929
Epoch 382/400
17939/17939 [==============================] - 2s 89us/step - loss: 0.1716 - accuracy: 0.9530 - val_loss: 0.3190 - val_accuracy: 0.9153

Epoch 00382: val_accuracy did not impro

In [18]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(model_history.history['loss'], color='b', label="Training loss")
ax1.plot(model_history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 400, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(model_history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(model_history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 400, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

# Model Analysis
Finding the Confusion matrix,Precision,Recall and F1 score to analyse the model thus created

In [19]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fig.savefig(os.path.join(MODEL_PATH,"confusion_matrix.png"))
    return fig

In [20]:
def print_heatmap(n_labels, n_predictions, class_names):
    labels = n_labels #sess.run(tf.argmax(n_labels, 1))
    predictions = n_predictions #sess.run(tf.argmax(n_predictions, 1))

#     confusion_matrix = sess.run(tf.contrib.metrics.confusion_matrix(labels, predictions))
    matrix = confusion_matrix(labels.argmax(axis=1),predictions.argmax(axis=1))
    row_sum = np.sum(matrix, axis = 1)
    w, h = matrix.shape

    c_m = np.zeros((w, h))

    for i in range(h):
        c_m[i] = matrix[i] * 100 / row_sum[i]

    c = c_m.astype(dtype = np.uint8)

    
    heatmap = print_confusion_matrix(c, class_names, figsize=(18,10), fontsize=20)

In [23]:
class_names = list()
for name,idx in labels_id.items():
    class_names.append(name)
# print(class_names)
ypred = VGG16_model.predict(valid_vgg16,verbose=1)

4485/4485 [==============================] - 0s 24us/step


In [26]:
print_heatmap(ytest,ypred,class_names)

# Precision Recall F1 Score

In [27]:
ypred_class = np.argmax(ypred,axis=1)
# print(ypred_class[:10])
ytest = np.argmax(ytest,axis=1)

In [30]:
accuracy = accuracy_score(ytest,ypred_class)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(ytest, ypred_class,average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(ytest,ypred_class,average='weighted')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(ytest,ypred_class,average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.915719
Precision: 0.916331
Recall: 0.915719
F1 score: 0.915800
